In [ ]:
!pip install matplotlib-venn

In [ ]:
pip install snscrape

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 2.1 MB/s eta 0:00:00


In [ ]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=afff21ed519581e2c84b51f3c48f197d99e8857db1ea0065707b5f7ff64f5a8d
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji


In [ ]:
import snscrape.modules.twitter as snt
import pandas as pd
import re
import emoji
from datetime import date, datetime
from collections import Counter
import nltk

In [ ]:
#Reading twitter dataset file for preprocessing

final=pd.read_csv("final.csv")
print(final.head(5))

   Unnamed: 0                                                url  \
0           0  https://twitter.com/gand_oliver/status/1648233...   
1           1  https://twitter.com/MirrorF1/status/1648233870...   
2           2  https://twitter.com/MotorsportWeek/status/1648...   
3           3  https://twitter.com/f1reader/status/1648233423...   
4           4  https://twitter.com/Racingnews365C/status/1648...   

                        date            id  \
0  2023-04-18 07:56:10+00:00  1.650000e+18   
1  2023-04-18 07:55:52+00:00  1.650000e+18   
2  2023-04-18 07:54:56+00:00  1.650000e+18   
3  2023-04-18 07:54:05+00:00  1.650000e+18   
4  2023-04-18 07:53:56+00:00  1.650000e+18   

                                                user  replyCount  \
0  {'username': 'gand_oliver', 'id': 753267774017...           0   
1  {'username': 'MirrorF1', 'id': 113622858, 'dis...           2   
2  {'username': 'MotorsportWeek', 'id': 27343080,...           0   
3  {'username': 'f1reader', 'id': 23044052

In [ ]:
#replacing "" with NA in retweetedTweets and quotedTweets columns
final.fillna({"retweetedTweet": "NA", "quotedTweet": "NA"}, inplace=True)

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

TwtwithoutAT=[]
sentimentScore=[]
lemm_tokens=[]
tfidf_matrices=[]
for tweets in final[['renderedContent'][0]]:
    #removing @
    tweets = re.sub("@+","",tweets) #@[A-Za-z0-9]+
    #removing hyperlinks
    tweets = re.sub("(?:\\@|http?\\://|https?\\://|www)\\S+","",tweets)
    #replacing # and _
    tweets = tweets.replace("#", "").replace("_", " ")
    #removing/replacing emojis
    tweets = emoji.replace_emoji(tweets,"")
    #removing spaces from start and end
    tweets = re.sub(r'\n','', tweets).strip()
    # Replace Unicode characters in the 'tweets' variable
    tweets = tweets.replace("\u2019", "'")
    tweets = tweets.replace("\u2018", "'")
    tweets = tweets.replace("\u2026", "...")
    tweets = tweets.replace("\u2013", "-")
    tweets = tweets.replace("\u201c", "\"")
    tweets = tweets.replace("\u201d", "\"")

    text = tweets

  # Tokenization
    tokens = word_tokenize(text)

  # Stop words removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]

    # TF-IDF
    documents = [" ".join(lemmatized_tokens)]  # Creating a single document for simplicity
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    #Appending the final output to the lists
    lemm_tokens.append(lemmatized_tokens)
    tfidf_matrices.append(tfidf_matrix)
    TwtwithoutAT.append(text)

    # Sentiment Analysis
    analyzer = SentimentIntensityAnalyzer()
    sentimentScore.append(analyzer.polarity_scores(text))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
#determining Sentiment values in categorical form

SentimentVal=[]

for i in sentimentScore:
    if i["compound"]>0:
        SentimentVal.append("Positive")
    elif i["compound"]<0:
        SentimentVal.append("Negative")
    else:
        SentimentVal.append("Neutral")

In [ ]:
final['RefinedContent']=TwtwithoutAT
final['lemmatized_tokens']=lemm_tokens
final['tfidf_matrix']=tfidf_matrices
final['SentimentValue']=SentimentVal

In [ ]:
final.to_csv("Twt_Refined_Final.csv")

In [ ]:
import pandas as pd

# Load the CSV file into the DataFrame
final = pd.read_csv("Twt_Refined_Final.csv")

# Separate numerical and categorical columns
numerical_cols = final.select_dtypes(include=['float64']).columns.tolist()
categorical_cols = final.select_dtypes(include=['object']).columns.tolist()

print("Numerical Columns:")
print(numerical_cols)

print("\nCategorical Columns:")
print(categorical_cols)



Numerical Columns:
['id', 'retweetedTweet']

Categorical Columns:
['url', 'date', 'user', 'lang', 'sourceLabel', 'quotedTweet', 'hashtags', 'renderedContent', 'RefinedContent', 'lemmatized_tokens', 'tfidf_matrix', 'SentimentValue']


In [ ]:

# Detect and handle outliers using IQR
for col in numerical_cols:
    Q1 = final[col].quantile(0.25)
    Q3 = final[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    final[col] = final[col].apply(lambda x: max(lower_bound, min(upper_bound, x)))

# Save the modified DataFrame
final.to_csv("Twt_Refined_Final_NoOutliers.csv", index=False)



In [ ]:

# Calculate the correlation matrix
correlation_matrix = final[numerical_cols].corr()

# Save the correlation matrix to a CSV file
correlation_matrix.to_csv("Correlation_Matrix.csv", index=False)


In [ ]:

# Specify the categorical columns you want to one-hot encode
categorical_cols = ['SentimentValue']

# Perform one-hot encoding
encoded_df = pd.get_dummies(final, columns=categorical_cols)

# Save the encoded DataFrame to a CSV file
encoded_df.to_csv("Encoded_Twt_Refined_Final.csv", index=False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Load the CSV file into the DataFrame
data = pd.read_csv("Encoded_Twt_Refined_Final.csv")

# Specify the columns you want to normalize (numerical columns)
numerical_cols = ['SentimentValue_Positive', 'SentimentValue_Neutral', 'SentimentValue_Negative']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max normalization to the specified numerical columns
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

# Save the normalized DataFrame to a new CSV file
data.to_csv("Normalized_Encoded_Twt_Refined_Final.csv", index=False)


In summary, the typical order is: Normalization -> Scaling -> Dimensionality Reduction (if needed).

However, keep in mind that the specific order might vary based on your data, the algorithms you're using, and the goals of your analysis. It's a good practice to experiment with different preprocessing orders and evaluate their impact on your specific task to determine the most suitable approach.